### topic -->blog generation


start --> gen outline --> gen blog --> END 

In [34]:
from langgraph.graph import StateGraph , START  , END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv  import load_dotenv

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

In [35]:
class Blog_state(TypedDict) : 
    title : str
    outline : str
    content : str
    rating : float


In [36]:
def create_outline(state : Blog_state)->Blog_state:
    title = state['title']
    prompt = f'Generate and outline for a blog on the topic  {title}'
    outline =model.invoke(prompt).content
    state['outline'] = outline
    return state

def create_blog(state : Blog_state)->Blog_state:
    title = state['title']
    outline = state['outline']
    prompt = f'write a detailed blog on the title {title} using the follwing outline :  {outline}'
    content = model.invoke(prompt).content
    state['content'] = content
    return state

def rate_blog (state : Blog_state)->Blog_state:
    outline = state['outline']
    blog = state['content']
    prompt = f'Based on the following outline  : {outline} rate this blog out of 10 only give me a float value not feedback needed: {blog}'
    rating = model.invoke(prompt).content
    state['rating'] = rating
    return state
    

In [37]:
graph = StateGraph(Blog_state)
graph.add_node('create_outline' , create_outline)
graph.add_node('create_blog' , create_blog )
graph.add_node('rate_blog' , rate_blog)
graph.add_edge(START , 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog','rate_blog')
graph.add_edge('rate_blog' , END)

workflow = graph.compile()


In [38]:
intial_state = {'title' : 'rise of AI ' }
final_state = workflow.invoke(intial_state)
final_state


{'title': 'rise of AI ',
 'outline': "## Blog Outline: The Rise of AI - A Transformative Era\n\n**I. Introduction (approx. 100 words)**\n\n* Hook: Start with a compelling anecdote or statistic showcasing AI's impact in everyday life (e.g., self-driving cars, personalized recommendations, medical diagnoses).\n* Briefly define Artificial Intelligence and its core components (machine learning, deep learning, natural language processing).\n* Thesis statement:  The rise of AI is rapidly transforming various aspects of society, presenting both immense opportunities and significant challenges.  This blog will explore key aspects of this transformation.\n\n\n**II. Key Drivers of AI's Rise (approx. 200 words)**\n\n* **Increased Computing Power:** Explain Moore's Law and its impact on AI development.  Mention the role of GPUs and specialized AI hardware.\n* **Big Data Availability:** Discuss the exponential growth of data and its crucial role in training AI models.  Mention sources like social m

In [40]:
print(final_state['rating'])

8.5
